In [4]:
import numpy as np
import nibabel as nib 
import os 
import pandas as pd
from utils.utils import * 
from utils.flips import get_prefered_xy
import math

In [5]:
"""
Export for the full brain fits is different: we do not make a model, we pick on sample ROI and export all the fitted values on that one target 
"""

rois_long = {'V1v': 1, 'V1d': 2, 'V2v': 3, 'V2d': 4, 'V3v': 5, 'V3d': 6, 'hV4': 7, 'VO-1': 8,
 'VO-2': 9, 'PHC-1': 10, 'PHC-2': 11, 'LO-1': 12, 'LO-2': 13, 'TO-1': 14, 'TO-2': 15
}
sior_long = {v: k for k,v in rois_long.items()}
to_reduce = {'V1v': 'V1', 'V1d': 'V1', 'V2v': 'V2', 'V2d': 'V2', 'V3v': 'V3', 'V3d': 'V3'}
sior = {v: k for k,v in rois.items()}

In [6]:
cos_sin = get_prefered_xy(subj_list, sessions, fetch_conds=False) # save that and open it matlab 


(907,)


In [7]:
cos_sin  #Y's sould probably be flipped

array([[ 1.        ,  0.        ],
       [ 0.6541633 ,  0.75635332],
       [ 0.36456779,  0.93117684],
       [ 0.53622597,  0.84407449],
       [ 0.68344992, -0.72999746],
       [ 0.72568947,  0.68802243],
       [ 0.427239  ,  0.90413874],
       [-0.29617682,  0.95513314]])

In [23]:

models_subs = {}

columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang"]

## Change this line to select which sampling space you want for the full brain model 
sampling_space = 'V1'
mode = "train"
rotated = True

#cos_sin = get_prefered_xy(subj_list) # save that and open it matlab 
#print(cos_sin)


for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    participant_df = pd.DataFrame(columns=columns)
    for k in range(5):
        fits_file = os.path.join(fits_dir, 'fits_inversed', sub,  f'fits_{sub}_{mode}_{sampling_space}_full_{k}.npy')

        chunk_df = pd.DataFrame(np.load(fits_file, allow_pickle=True, mmap_mode='r'), columns=columns)
        
        participant_df = pd.concat([participant_df, chunk_df], ignore_index=True)
        
    models_subs[sub] = participant_df
  #  models_subs[sub][columns[:-2]] = models_subs[sub][columns[:-2]].astype(np.float32)
        # this should be okay 
    models_subs[sub]['x0_prefered'] = models_subs[sub]['x0'] * cos_sin[i, 0]-models_subs[sub]['y0'] * cos_sin[i, 1] # the mds have different dimension -> so problem 
    models_subs[sub]['y0_prefered'] = models_subs[sub]['x0'] * cos_sin[i, 1]+models_subs[sub]['y0'] * cos_sin[i, 0]

    sub_m_study = models_subs[sub]
hier_index = pd.MultiIndex.from_arrays(
    [
        np.tile(list(rois.keys()), len(models_subs)),
        np.repeat(list(models_subs.keys()), len(rois.keys())),
    ],
    names =["roi", "sub"]
)
models_subs["subj01"]["test_var_explained"]

# check distribution of values in VO-1s for common conditions 

models_subs

{'subj01':               x0        y0     sigma      slope  intercept  \
 0       0.372415 -0.623870  4.931022  19.043626 -18.823531   
 1       0.418566 -0.594732  5.470514  20.087506 -19.886948   
 2       0.502126 -0.546833  5.711357  17.728630 -17.547676   
 3       0.349843 -0.618175  4.944373  19.525256 -19.306246   
 4       0.331911 -0.591486  5.757319  26.898727 -26.681219   
 ...          ...       ...       ...        ...        ...   
 453599  1.046572  0.558521  0.273111   0.118829  -0.008340   
 453600  0.899774 -0.317330  0.093094   0.326752  -0.026642   
 453601  1.049905 -0.773981  2.377122   0.124015  -0.119264   
 453602 -0.248348  1.050000  0.418259   0.040842  -0.007440   
 453603  1.049992 -0.589168  2.329143   0.152949  -0.149802   
 
         test_var_explained  var_explained   mds_ecc   mds_ang  x0_prefered  \
 0                 0.000685       0.039975  0.701712  2.603424     0.372415   
 1                -0.000799       0.036249  0.727258  2.528328     0.41856

In [24]:
hemis = ['lh', 'rh']
fields = columns
fields.append('x0_prefered')
fields.append('y0_prefered')
if rotated:
    rotation = "rotated"
if not rotated: 
    rotation = "notrotated"


for sub in subj_list:
        m = models_subs[sub]
        print(f'------ EXPORTING MODEL FOR {sub}')
        for f in fields:
            print(f'\t{f}')
            s = -1
            for hemi in hemis:
                print(f'\t\t{hemi}')
                
                
                maskdata_long_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.{sub}.nans_full.testrois.mgz') # should be the right mask
                maskdata_long = nib.load(maskdata_long_file).get_fdata().squeeze()
                
                print(maskdata_long.shape)
        
                maskdata_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.short.reduced.{sub}.testrois.npy')
                maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)

                data_out_file = os.path.join(results_dir, sub, f'{hemi}.{f}.full{sampling_space}.mgz')
                label_out_file = os.path.join(label_dir,'freesurfer', sub, 'label', f'{hemi}.{f}.full{sampling_space}.mgz') # annoying but necessary for matlab output
                if os.path.exists(data_out_file):
                    pass
                data_out = np.zeros(maskdata_long.shape)
             
                wrongs = []
                for i in range(maskdata_long.shape[0]):
                    voxel = int(maskdata_long[i])
                    if voxel ==-100:
                      
                        continue
                    s += 1
                #    if sior_long[voxel] in to_reduce.keys():
                 #       roi = rois[to_reduce[sior_long[voxel]]]
                 #   else:
                 #       roi = rois[sior_long[voxel]]
                  #  roi = rois[sior[voxel]]
                    # prove that the order is maintained
                #    if roi != rois[m.iloc[s].roi]:
                 #       print(f'{i}: {roi} != {rois[m.iloc[s].roi]}')
                  #      wrongs.append(i)
                  #  if f in ['roi', 'best_roi']:
                   #     data_out[i] = rois[m.iloc[s][f]]
                    if f == 'mds_ang':
                        data_out[i] = math.degrees(m.iloc[s][f]) + 180
                    elif f == 'intercept':
                        data_out[i] = m.iloc[s][f] + abs(m[f].min()) + 1e-08
                    elif f in ['x0', 'y0', 'x0_prefered', 'y0_prefered']:
                        data_out[i] = m.iloc[s][f] + 1.1
                    elif f in ['var_explained', 'test_var_explained']:
                        data_out[i] = m.iloc[s][f]  + 1 # Add 1 for the variance explain, so we can still mask the non relevant value and plot the "negative" variance
                    else:
                        data_out[i] = m.iloc[s][f]  
                img = nib.Nifti1Image(np.expand_dims(data_out, axis=(1, 2)), np.eye(4))

                nib.loadsave.save(img, data_out_file)
                nib.loadsave.save(img, label_out_file)
                if f == 'mds_ang':
                    for ang in [45, 90, 135, 180, 225, 270, 305]:
                        data_out_file = os.path.join(results_dir, sub, f'{hemi}.{f}.{str(ang)}.full{sampling_space}.mgz')
                        label_out_file = os.path.join(label_dir, 'freesurfer', sub, 'label', f'{hemi}.{f}.{str(ang)}.full{sampling_space}.mgz')
                        data_out_ang = np.copy(data_out)
                        data_out_ang[data_out != 0] += ang
                        data_out_ang = data_out_ang % 360
                        img = nib.Nifti1Image(np.expand_dims(data_out_ang, axis = (1, 2)), np.eye(4))
                        nib.loadsave.save(img, data_out_file)
                        nib.loadsave.save(img, label_out_file)


print('end')
                        
            

------ EXPORTING MODEL FOR subj01
	x0
		lh
(227021,)


/home/harveylab/miniconda3/envs/thesis/lib/python3.10/site-packages/nibabel/spatialimages.py:558: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)


		rh
(226601,)
	y0
		lh
(227021,)
		rh
(226601,)
	sigma
		lh
(227021,)
		rh
(226601,)
	slope
		lh
(227021,)
		rh
(226601,)
	intercept
		lh
(227021,)
		rh
(226601,)
	test_var_explained
		lh
(227021,)
		rh
(226601,)
	var_explained
		lh
(227021,)
		rh
(226601,)
	mds_ecc
		lh
(227021,)
		rh
(226601,)
	mds_ang
		lh
(227021,)
		rh
(226601,)
	x0_prefered
		lh
(227021,)
		rh
(226601,)
	y0_prefered
		lh
(227021,)
		rh
(226601,)
------ EXPORTING MODEL FOR subj02
	x0
		lh
(239633,)
		rh
(239309,)
	y0
		lh
(239633,)
		rh
(239309,)
	sigma
		lh
(239633,)
		rh
(239309,)
	slope
		lh
(239633,)
		rh
(239309,)
	intercept
		lh
(239633,)
		rh
(239309,)
	test_var_explained
		lh
(239633,)
		rh
(239309,)
	var_explained
		lh
(239633,)
		rh
(239309,)
	mds_ecc
		lh
(239633,)
		rh
(239309,)
	mds_ang
		lh
(239633,)
		rh
(239309,)
	x0_prefered
		lh
(239633,)
		rh
(239309,)
	y0_prefered
		lh
(239633,)
		rh
(239309,)
------ EXPORTING MODEL FOR subj03
	x0
		lh
(240830,)
		rh
(243023,)
	y0
		lh
(240830,)
		rh
(243023,)